In [1]:
from IPython.display import Image
import matplotlib.pyplot as plt #for showing the image
import matplotlib.image as mpimg #for reading the image
import numpy as np 

#   <h1><center>BRZO MNOŽENJE MATRICA</center></h1>
## <h1><center>SEMINAR</center></h1>\\
<h1><center>Numerička linearna algebra</center></h1>
Studenti : Josipa Despotović, Magdalena Tomašević

# UVOD <br>
<b>Koliko brzo možemo pomnožiti dvije matrice?</b> <br><br>
Množenje matrica jedna je od osnovnih operacija u raznim algoritmima numeričke analize. Temeljni problem u teorijskoj računalnoj znanosti je upravo određivanje vremenske složenosti matričnog množenja, jedne od najosnovnijih linearnih algebarskih operacija. Upravo zbog njene važnosti dan danas se ulaže veliki trud u razvijanje algoritama za što brže i efikasnije množenje matrica. Primjene matričnog množenja u računskim problemima se očituju u raznim poljima uključujući znanstveno računanje i prepoznavanje uzoraka te naizgled nepovezane probleme kao što je brojanje staza u grafu. Matrično množenje igra važnu ulogu i u fizici, inženjerstvu, računalnoj znanosti i drugim područjima te se koristi kao potprogram u mnogim računalnim problemima.
Mnoštvo algoritama za množenje matrica je dizajnirano na raznim tipovima hardware-a uključujući paralelne i distribuirane sustave, gdje se rad na računanju raspodjeljuje na više procesora ( čak i više mreža).<br><br>
Postavlja se pitanje : 
<b>Koji je najbrži algoritam za umnožavanje matrice?</b> <br><br>
Pitanje se obično prevodi u određivanje eksponenta matričnog množenja, tj. <b> najmanjeg realnog broja $\omega$ </b> takvog da produkt dvije matrice dimenzija $n \times m$ , nad poljem F, može biti dobiven koristeći $n^{\omega}$ operacija nad poljem F.<br>
Mnogo je rada uloženo u stvaranje učinkovitih algoritama množenja matrica tijekom godina, ali je granica još uvijek između $2\leq\omega\leq3$.



U ovom seminaru donosimo povijesni pregled i razvoj brzih algoritama za množenje matrica.

# 1.	Iterativni algoritam (naivna metoda)<br>
Za dane dvije matrice A i B, dimenzija $n \times m$ i $m \times p$ redom, $A=(a_{ik})$ i $B=(b_{kj})$ želimo izračunati njihov produkt matricu  $C=(c_{ij})$, dimenzije $n \times p$. Vrijednosti $c_{ij}$ su dane u obliku bilinearne forme 
$c_{ij}=\sum_{k=1}^{m} a_{ik}b_{kj},1 ≤i,j≤m $ .
Iz toga se može konstruirati jednostavan algoritam koji se ide po indeksima i od 1 do n i j od 1 do p, izračunavajući umnožak  pomoću ugniježđene petlje : <img src="slike/pic1.png" width="400" height="600" />


In [2]:
# Program za množenje 2 matrice koristeći ugniježđene petlje

# 3x3 matrix
X = [[12,7,3],
    [4 ,5,6],
    [7 ,8,9]]
# 3x4 matrix
Y = [[5,8,1,2],
    [6,7,3,0],
    [4,5,9,1]]
# rezultat je 3x4
result = [[0,0,0,0],
         [0,0,0,0],
         [0,0,0,0]]

# kroz retke od X
for i in range(len(X)):
   # kroz stupce od Y
   for j in range(len(Y[0])):
       # kroz retke od Y
       for k in range(len(Y)):
           result[i][j] += X[i][k] * Y[k][j]

for r in result:
   print(r)

[114, 160, 60, 27]
[74, 97, 73, 14]
[119, 157, 112, 23]


Navedeni algoritam je složenosti $\Theta(nmp)$. Često se ,radi jednostavnosti analize algoritma, pretpostavlja da su ulazne matrica A i B kvadratne, dimenzija $n \times n$ te je u tom slučaju složenost algoritma $\Theta(n^3)$.
## <b>Cache memorija: </b>
Tri petlje u iterativnom množenju matrica mogu se proizvoljno zamijeniti jedna s drugom bez utjecaja na točnost ili asimptotsko vrijeme rada. Međutim, redoslijed može imati značajan utjecaj na praktičnu izvedbu zbog načina pristupa memoriji i korištenja algoritma za cache memoriju. 
Koji je redoslijed najbolji ovisi i o tome jesu li matrice pohranjene u redoslijedu row-major ili column-major ili mješavina oba.<br>
Konkretno, u idealiziranom slučaju potpuno asocijativnog cache-a koji se sastoji od M cache linija od b bajta svaki, gornji algoritam je sub-optimalan za A i B pohranjene u redoslijedu row-major. Kada je $n>\frac{M}{b}$ , svaka iteracija unutarnje petlje (istovremeni prolaz kroz red u A i stupac u B) izaziva propust u predmemoriji prilikom pristupa elementu B. <img src="slike/pic2.png" width="200" height="400" />


To znači da algoritam prouzrokuje promašaj predmemorije $\Theta(n^3)$ u najgorem slučaju. Od 2010. brzina memorije u usporedbi s procesorima je takva da propusti u memoriji, a ne stvarni proračuni, dominiraju vremenom rada za velike matrice.<br>
Optimalna varijanta iterativnog algoritma za A i B u row-major redoslijedu je "popločana" verzija, gdje je matrica implicitno podijeljena na kvadratne pločice veličine $\sqrt{M}  \times \sqrt{M}$ : <img src="slike/pic3.PNG" width="600" height="700" /> <br>
U idealnom cache modelu, ovaj algoritam radi samo $\Theta(\frac{n^3}{(b\sqrt{M})})$ cache promašaja.

# 2.	Podijeli pa vladaj algoritam <br>
Podijeli pa vladaj algoritam je alternativa iterativnom algoritmu za množenje matrica. On se temelji na rastavljanju matrica na blokove,npr. : <br>

$P=\begin{pmatrix} 1 & 1 & 2 & 2\\ 1&1&2&2\\ 3&3&4&4\\3&3&4&4 \end{pmatrix}$   možemo rastaviti na četiri bloka dimenzije $ 2 \times 2$ <br> <br>
<center>$P_{11}=\begin{pmatrix} 1 & 1\\ 1 & 1 \end{pmatrix}$ , $P_{12}=\begin{pmatrix}2 & 2\\ 2 & 2 \end{pmatrix}$ ,$P_{21}=\begin{pmatrix}3 & 3\\ 3 & 3 \end{pmatrix}$ , $P_{22}=\begin{pmatrix}4 & 4\\ 4 & 4 \end{pmatrix}$ </center><br>
te radi za sve kvadratne matrice čije su dimenzije potencije broja 2, $2^n \times 2^n$, za neki n.<br><br>
Stoga, za matrice<br>
$$C=\begin{pmatrix} c_{11} & c_{12}\\ c_{21} & c_{22} \end{pmatrix} , A=\begin{pmatrix} a_{11} & a_{12}\\ a_{21} & a_{22} \end{pmatrix}, B=\begin{pmatrix} b_{11} & b_{12}\\ b_{21} & b_{22} \end{pmatrix} $$ <br>

definiramo produkt kao <br><br>
$$C=\begin{pmatrix} c_{11} & c_{12}\\ c_{21} & c_{22} \end{pmatrix} = \begin{pmatrix} a_{11} & a_{12}\\ a_{21} & a_{22} \end{pmatrix}\begin{pmatrix} b_{11} & b_{12}\\ b_{21} & b_{22} \end{pmatrix} = \begin{pmatrix} a_{11}b_{11}+a_{12}b_{21} & a_{11}b_{12}+a_{12}b_{22}\\ a_{21}b_{11}+a_{22}b_{21} & a_{21}b_{12}+a_{22}b_{22} \end{pmatrix}$$ <br>
Sastoji se od 8 množenja parova blok matrica te 4 zbrajanja. <br>
Poznavajući te jednadžbe, sada izrađujemo jednostavan, rekurzivni algoritam koji izvršava manje množenja, koristeći skalarno množenje $c_{11}=a_{11} b_{11}$ kao svoj bazni slučaj.<br>
<img src="slike/pic8.PNG" width="500" height="600" /> <br>


Analizirajmo navedeni algoritam: <br>
Neka je T(n) vrijeme izvršavanja ovog algoritma.<br>
U osnovnom slučaju, kada je broj redaka jednak 1, algoritam izvodi samo jedno skalarno množenje.<br>
U slučaju kada je n > 1, ukupno vrijeme se izračunava kao zbroj vremena particioniranja (dijeljenje matrica na 4 dijela), vremena za sve rekurzivne pozive i vremena dodavanja matrica koje proizlaze iz rekurzivnih poziva( obračunavanje osam rekurzivnih poziva na matricama veličine $n/2$ i $Θ(n^2 )$ za zbroj 4 para rezultantnih matrica).
Složenost ovog algoritma kao funkcije n dana je sa <br><br>

<center>$T(1)=Θ(1)$ , za n = 1</center> <br>
<center>$T(n)=8T(n/2)+Θ(n^2)$, za n > 1</center> .<br>
Ukupna složenost je <br>
<center>$T(n)=Θ(8^{log_{2}n})=Θ(n^3)$ </center> <br>

pa ova rekurzija ima rješenje kao i iterativni algoritam. <b>Gdje je napredak?</b>

## Ne-kvadratne matrice <br>

Varijanta ovog algoritma koja radi za matrice proizvoljnih oblika i i koja je brža u praksi razdvaja matrice u dvije umjesto četiri podmatrice, kako slijedi.
Podjela matrice sada znači da se ona dijeli na dva dijela jednake veličine ili što je moguće bliže jednakim veličinama u slučaju čudnih dimenzija.<br>
<img src="slike/pic5.PNG" width="600" height="700" />

## Cache memorija:<br>

Stopa propuštanja predmemorije rekurzivnog množenja matrice je ista kao i kod popločane iterativne verzije, ali za razliku od tog algoritma, rekurzivni algoritam je cache nesvjestan,neosjetljiv: nema parametra podešavanja potrebnog za dobivanje optimalne performanse predmemorije, i ponaša se dobro u multiprogramskom okruženju gdje su veličine cachea efektivno dinamične zbog drugih procesa koji zauzimaju cache prostor. (Jednostavan iterativni algoritam također je cache nesvjestan, ali u praksi je mnogo sporiji ako izgled matrice nije prilagođen algoritmu.)<br>

Broj promašaja predmemorije nastalih ovim algoritmom, na stroju s M redova idealne predmemorije, svaki veličine b bajta, ograničen je sa <br>
<h2><center>
$Θ(m+n+p+\frac{mn+np+mp}{b} +\frac{mnp}{b\sqrt{M}})$ </center></h2>

# 3. Strassenov algoritam <br>

Postoje algoritmi koji pružaju bolje vrijeme rada od jednostavnih. Prvi koji je otkriven bio je Strassenov algoritam, kojeg je 1969. godine osmislio Volker Strassen i često nazivan "brzo množenje matrica".<br>
Temelji se na načinu množenja dviju 2 × 2 matrica koje zahtijeva samo 7 množenja (umjesto uobičajenih 8), na račun nekoliko dodatnih operacija sabiranja i oduzimanja.<br> To je dovoljno da se smanji složenost vremena izvođenja na sub-kubično vrijeme!
Primjena ove metode rekurzivno daje algoritam s višestrukim troškovima $O(n^{log_{2}7})≈O(n^{2.807})$.<br>
Strassenov algoritam je složeniji, a numerička stabilnost je smanjena u usporedbi s naivnim algoritmom, nad domenama kao što su konačna polja, gdje numerička stabilnost nije problem.<br><br>
U Strassenovom algoritmu imamo , dakle, 1 rekurzivni poziv manje, ali imamo nekoliko novih zbrajanja matrica dimenzije $\frac{n}{2} \times \frac{n}{2}$. Algoritam se sastoji od 4 koraka :<br><br>
<b>1)</b> Podijeli ulazne matrice A i B na $\frac{n}{2} \times \frac{n}{2}$ blok matrice --> potrebno vrijeme $Θ(1)$ <br><br>
<b>2)</b> Stvori 10 matrica $S_{1},S_{2},...,S_{10}$ od kojih je svaka suma ili razlika dvije matrice iz koraka 1), tj: <br><br>
<img src="slike/pic9.png" width="200" height="300" /><br>
Potrebno vrijeme za stvaranje navedenih 10 matrica je $Θ(n^2)$ <br><br>
<b>3)</b> Koristeći blok matrice kreirane u koraku 1) i 2) , rekurzivno izračunaj 7 matrica $P_{1},...,P_{7}$, od kojih je svaka dimenzije  $\frac{n}{2} \times \frac{n}{2}$.<br>
<img src="slike/pic10.png" width="500" height="600" /><br><br>
<b>4)</b> Dodavajući i oduzimajući blok matrice P dobijemo <br><br>
<img src="slike/pic12.png" width="200" height="300" /><br>
Te četiri matrice možemo izračunati za $Θ(n^2)$.<br>
Dakle, Strassenov algoritam glasi <br><br>
<img src="slike/pic13.jpg" width="600" height="800" /><br>

Strassen je zamijenio jedno množenje matrice za konstantan broj matričnih zbrajanja, tako da je dobio manje asimptotsko vrijeme rada.Imamo,<br><br>
<center>$T(n)= 7*T(\frac{n}{2})+Θ(n^2) => T(n)=O(n^{log_{2}7})$</center> <br>
<center> tj. $\omega \leq log_{2}7 = 2.807...$ <br><br>

Donosimo implementaciju Strasenova algoritma za matrice dimenzija $2^n \times 2^n$.

In [1]:
import random
random.seed(1234)

def createRandomMatrix(n):
    maxVal = 1000 # elementi do 1000
    matrix = []
    for i in range(n):
        matrix.append([random.randint(0,maxVal) for el in range(n)])
    return matrix

def saveMatrix(matrixA, matrixB, filename): # za spremanje matrice
    f = open(filename, 'w')
    for i, matrix in enumerate([matrixA, matrixB]):
        if i != 0:
            f.write("\n")
        for line in matrix:
            f.write("\t".join(map(str, line)) + "\n")
n = 128 #potencija broja 2
matrixA = createRandomMatrix(n)
matrixB = createRandomMatrix(n)
#saveMatrix(matrixA, matrixB, "mat.csv")
#print(matrixA)

In [2]:
#a = [[1,1,1,1],[2,2,2,2],[3,3,3,3],[4,4,4,4]] za provjeru
#b = [[5,5,5,5],[6,6,6,6],[7,7,7,7],[8,8,8,8]]

def new_m(p, q): # stvorimo matricu ispunjenu s nulama
    matrix = [[0 for row in range(p)] for col in range(q)]
    return matrix

def Pomnozi(a, b): # obično množenje 2 matrice
    if len(a[0]) != len(b): # if # of col != # of rows:
        return "Matrices are not m*n and n*p"
    else:
        p_matrix = new_m(len(a), len(b[0]))
        for i in range(len(a)):
            for j in range(len(b[0])):
                for k in range(len(b)):
                    p_matrix[i][j] += a[i][k]*b[k][j]
    return p_matrix

def split(matrix): # podijelimo matricu na četvrtine
    a = matrix
    b = matrix
    c = matrix
    d = matrix
    while(len(a) > len(matrix)/2):
        a = a[:len(a)//2]
        b = b[:len(b)//2]
        c = c[len(c)//2:]
        d = d[len(d)//2:]
    while(len(a[0]) > len(matrix[0])/2):
        for i in range(len(a[0])//2):
            a[i] = a[i][:len(a[i])//2]
            b[i] = b[i][len(b[i])//2:]
            c[i] = c[i][:len(c[i])//2]
            d[i] = d[i][len(d[i])//2:]
    return a,b,c,d

def add_m(a, b):
    if type(a) == int:
        d = a + b
    else:
        d = []
        for i in range(len(a)):
            c = []
            for j in range(len(a[0])):
                c.append(a[i][j] + b[i][j])
            d.append(c)
    return d

def sub_m(a, b):
    if type(a) == int:
        d = a - b
    else:
        d = []
        for i in range(len(a)):
            c = []
            for j in range(len(a[0])):
                c.append(a[i][j] - b[i][j])
            d.append(c)
    return d


def strassen(a, b, q):
    # osnovni slučaj : 1x1 matrica
    if q == 1:
        d = [[0]]
        d[0][0] = a[0][0] * b[0][0]
        return d
    else:
        # podijelimo matricu na četvrtine
        a11, a12, a21, a22 = split(a)
        b11, b12, b21, b22 = split(b)

        # p1 = (a11+a22) * (b11+b22)
        p1 = strassen(add_m(a11,a22), add_m(b11,b22), q/2)

        # p2 = (a21+a22) * b11
        p2 = strassen(add_m(a21,a22), b11, q/2)

        # p3 = a11 * (b12-b22)
        p3 = strassen(a11, sub_m(b12,b22), q/2)

        # p4 = a22 * (b21-b11)
        p4 = strassen(a22, sub_m(b21,b11), q/2)

        # p5 = (a11+a12) * b22
        p5 = strassen(add_m(a11,a12), b22, q/2)

        # p6 = (a21-a11) * (b11+b12)
        p6 = strassen(sub_m(a21,a11), add_m(b11,b12), q/2)

        # p7 = (a12-a22) * (b21+b22)
        p7 = strassen(sub_m(a12,a22), add_m(b21,b22), q/2)


        # c11 = p1 + p4 - p5 + p7
        c11 = add_m(sub_m(add_m(p1, p4), p5), p7)

        # c12 = p3 + p5
        c12 = add_m(p3, p5)

        # c21 = p2 + p4
        c21 = add_m(p2, p4)

        # c22 = p1 + p3 - p2 + p6
        c22 = add_m(sub_m(add_m(p1, p3), p2), p6)

        c = new_m(len(c11)*2,len(c11)*2)
        for i in range(len(c11)):
            for j in range(len(c11)):
                c[i][j]                   = c11[i][j]
                c[i][j+len(c11)]          = c12[i][j]
                c[i+len(c11)][j]          = c21[i][j]
                c[i+len(c11)][j+len(c11)] = c22[i][j]

        return c

print ("Strassen-ov rezultat :")
print (strassen(matrixA, matrixB, 128))
print ("Rezultat običnog množenja :")
print (Pomnozi(matrixA, matrixB))

Strassen-ov rezultat :
[[30085144, 30498974, 27049160, 31786152, 28357462, 29459240, 28012355, 27744764, 29380407, 28753450, 27621616, 30690630, 29579757, 29149892, 30720162, 28259583, 28067334, 31758085, 29498290, 27099240, 31082243, 30581946, 28759580, 29824982, 30160606, 32897305, 29212578, 30718443, 31898533, 28939240, 27976545, 29069898, 24761226, 30917646, 27003190, 28965253, 30188253, 26755114, 29796510, 31958928, 26169324, 26680651, 32445247, 30231901, 27168616, 29582203, 28929568, 29349461, 28709749, 29234697, 28244510, 28772036, 31286073, 30113731, 30625419, 29434396, 26900197, 27270800, 27115558, 28687922, 27469437, 29383110, 27524807, 30636559, 26812582, 33602471, 27821303, 27424731, 30035370, 29052711, 28162176, 26151607, 26053712, 25268057, 28532624, 29965935, 29974584, 32153166, 28626748, 32148488, 23484770, 31448470, 26572395, 25549668, 29396450, 30538113, 32844197, 26146398, 30471664, 29215639, 29511645, 31442934, 28744605, 28144857, 32136669, 27499972, 28309095, 29092

[[30085144, 30498974, 27049160, 31786152, 28357462, 29459240, 28012355, 27744764, 29380407, 28753450, 27621616, 30690630, 29579757, 29149892, 30720162, 28259583, 28067334, 31758085, 29498290, 27099240, 31082243, 30581946, 28759580, 29824982, 30160606, 32897305, 29212578, 30718443, 31898533, 28939240, 27976545, 29069898, 24761226, 30917646, 27003190, 28965253, 30188253, 26755114, 29796510, 31958928, 26169324, 26680651, 32445247, 30231901, 27168616, 29582203, 28929568, 29349461, 28709749, 29234697, 28244510, 28772036, 31286073, 30113731, 30625419, 29434396, 26900197, 27270800, 27115558, 28687922, 27469437, 29383110, 27524807, 30636559, 26812582, 33602471, 27821303, 27424731, 30035370, 29052711, 28162176, 26151607, 26053712, 25268057, 28532624, 29965935, 29974584, 32153166, 28626748, 32148488, 23484770, 31448470, 26572395, 25549668, 29396450, 30538113, 32844197, 26146398, 30471664, 29215639, 29511645, 31442934, 28744605, 28144857, 32136669, 27499972, 28309095, 29092786, 27871961, 28174247

Strassenov algoritam nije osjetljiv na cache memoriju. Analiza algoritma ponašanja predmemorije pokazala je da nastaje $ Θ(1+\frac{n^2}{b}+\frac{n^{log_2 7}}{b\sqrt{M}})$ cache propusta tijekom izvršenja, pretpostavljajući idealiziranu predmemoriju od M redaka, svaki od b byte-ova.<br><br>

No, ipak općenito u praksi Strassenova metoda nije poželjna za praktične primjene iz sljedećih razloga: <br>
<ol>
  <li>
Konstante korištene u Strassenovoj metodi su visoke (veće od konstantnih faktora u O(n^3) i za tipičnu primjenu naivna metoda radi bolje. </li>
  
  <li>Za rijetke matrice (matrice u kojima je većina elemenata jednaka 0), postoje puno bolji algoritmi posebno dizajnirani za tu vrstu matrica. </li>
   <li>Blok matrice u rekurziji zahtijevaju dodatan prostor.</li>
   <li>Zbog ograničene preciznosti računalne aritmetike na vrijednostima koje nisu cijeli brojevi, veće se pogreške gomilaju u Strassenovu algoritmu nego u naivnoj metodi.</li>
</ol>


# 4. Evolucija $\omega$ do vrijednosti 2<br>
Strassenovo nevjerojatno otkriće izazvalo je dugu liniju istraživanja koja su postupno smanjivala eksponent množenja matrice ω tijekom vremena. Gotovo cijelo desetljeće nakon otkrića nije postignut daljnji napredak.<br><br>
Pogledajmo neke osnovne tehnike koje su se koristile s ciljem poboljšanja množenja matrica.<br>
<img src="slike/pic18.png" width="600" height="800" /><br>
<b> BILINEARNI ALGORITMI I RANG TENZORA:</b> <br><br>
Bilinearni algoritam za množenje matrica je algebarski algoritam oblika :
<br>
1. Izvrši <center> $m_1=$(linearne kombinacije $a_{ij}$ -eva)*(linearne kombinacije $b_{ij}$ -eva)<br>
<b>.<br>
.<br>
.</b><br>
 $m_{t}=$(linearne kombinacije $a_{ij}$ -eva)*(linearne kombinacije $b_{ij}$ -eva)

</center> <br>
gdje je t bilinearna složenost algoritma.<br>
2. Svaki $c_{ij}$ je dobiven iz linearnih kombinacija $m_1,...m:t$.<br>
Sa $C^{bil}(n)$ označimo bilinearnu složenost najboljeg bilinearnog algoritma koji računa produkt dvije kvadratne matrice.<br> Tada vrijedi propozicija:<br><br>
<b>Propozicija 1.</b>: Neka su m i t pozitivni cijeli brojevi.Pretpostavimo da postoji bilinearni algoritam koji računa produkt dvije matrice dimenzija $m \times m$ sa bilinearnom složenošću t. Tada je $\omega \leq log_{m}(t)$.<br>
Ukratko, $C^{bil}(m) \leq t \implies \omega \leq log_{m}(t)$
<br><br>
U fizici i matematici, <b>tenzor</b> je poopćenje skalara i vektora, te se, poput vektora, sastoji od više skalarnih vrijednosti, samo što iznos indeksa potrebnog da se jednoznačno odredi na koju skalarnu vrijednost se misli, može biti različit od jedan, kao što je slučaj kod vektora.<br>
<img src="slike/pic20.png" width="600" height="800" /><br>
<b> Rang</b> je tada, u oznaci <b>R($<m,n,p>$)</b> minimalan t takav da $<m,n,p>$ možemo zapisati kao sumu od t uvjeta u obliku (linearne kombinacije $a_{ij}$ -eva)$\times$(linearne kombinacije $b_{ij}$ -eva)$\times$(linearne kombinacije $c_{ij}$ -eva). Vrijedi <b>R($<m,n,p>$) $\leq mnp$</b> te se dobije da je zapravo <b>rang jednak bilinearnoj složenosti</b>.Dakle, sada imamo R($<m,m,m>$)$\leq \implies m^{\omega}\leq t$.<br>
Donosimo još jedan bitan teorem :<br>
<img src="slike/pic21.png" width="600" height="800" /><br>

<br>
Godine 1978. Pan je pronašao eksplicitne algoritme za daljnje smanjenje ω pomoću tehnike koja se zove trilinearna agregacija. On je primjetio da je problem matričnog množenja ekvivalentan (uz pretpostavku da sve matrice imaju isti rang) pronalasku traga produkta triju matrica A,B i C.   Koristeći ovu tehniku te navedeni teorem, Pan je pokazao da možemo ubrzati množenje dvije 70 × 70 matrice u 143640 operacija.Rezultat je bio $\omega\leq log_{70}143640 < 2.79$.<br><br>

<b> GRANIČNI RANG TENZORA </b> <br><br>

Promotrimo sada opću definiciju tenzora nad vektorskim prostorima U, V i W nad poljem F te definiciju ranga takvog tenzora.<br>
<img src="slike/pic22.png" width="500" height="600" /><br>
<img src="slike/pic23.png" width="500" height="600" /><br>
Granični rang tenzora definiramo na način :<br>
<img src="slike/pic24.png" width="500" height="600" /><br>

Očito je granični rang tenzora manji ili jednak rangu tenzora, za bilo koji tenzor T te se pokaže da je <b>granični rang jednak složenosti aproksimativnih bilinearnih algoritama.</b><br><br>
Godine 1979. Bini (i suradnici) prikazao je da se broj operacija potrebnih za izvođenje matričnog množenja može smanjiti razmatranjem bilinearnih algoritama i graničnog ranga tenzora. <br>
Konstruirali su tenzor nanačin :<br>
<img src="slike/pic25.png" width="500" height="600" /><br> 
te dobili da je  granični rang tako konstruiranog tenzora jednak 5.<br>
Bini je koristio 10 produkata kao ulaz za množenje matrica dimenzije $2 \times 3$ i $3 \times 3$ te je dobio :
<img src="slike/pic28.png" width="500" height="600" /><br> 
<img src="slike/pic27.png" width="500" height="600" /><br> 

<br><br>

<b> OSTALE METODE </b>

Godine 1981. Schonhage je razvio sofisticiranu teoriju koja uključuje bilinearnu složenost umnožavanja pravokutne matrice koja je pokazala da su približni bilinearni algoritmi još moćniji. Svojim teoremom nejednakosti asimptotske sume, on je dokazao $ω <2.55$. To je bilo veliko poboljšanje u odnosu na prethodnu granicu koju je dobio Bini.<br><br>

U sljedećim godinama Romani, Pan, Coppersmith i Winograd učinili su daljnja poboljšanja i postigli $ω <2.50$.<br><br>


Strassen je 1986. godine uveo svoju lasersku metodu koja daje $ω \leq 2.48.$ 
Ova metoda temelji se na uzimanju visokih tenzorskih potencija fiksnog tenzora, a zatim ih degenerira na disjunktnu sumu tenzora množenja matrica.<br><br>


Godine 1989. Coppersmith i Winograd su koristili ovu metodu kako bi ostvarili “svjetski rekord” za $ω <2.38$.
Algoritam Coppersmith – Winograd često se koristi kao građevni blok u drugim algoritmima kako bi se dokazale teoretske vremenske granice. Međutim, za razliku od Strassenovog algoritma, on se ne koristi u praksi jer samo daje prednost za jako velike matrice koje se ne mogu obraditi modernim hardverom. Algoritam Coppersmith Winograd bio je svjetski prvak 20 godina dok ga Stothers napokon nije poboljšao 2010.godine.
<br>

Neovisno o tome, Vassilevska-Williams je kombinirajući matematički prečac iz Stothersovog rada s vlastitim spoznajama i automatiziranom optimizacijom na računalima, ostvarila daljnje poboljšanje u 2012. godini, a Le Gall je 2014. godine usavršio svoje metode za dobivanje trenutnog svjetskog prvaka, $ω <2.3728639$.
<img src="slike/pic14.png" width="600" height="800" />
<br><br>

Le Gall algoritam i Coppersmith – Winograd algoritam, slični su Strassenovu algoritmu: osmišljen je način za množenje dvije $k\times k$ -matrice s manje od $k^3$ množenja, a ova se tehnika primjenjuje rekurzivno. Međutim, konstantni koeficijent skriven notacijom O(veliko O) toliko je velik da se korištenje ovih algoritama isplati samo za matrice koje su prevelike za rad na današnjim računalima.<br><br> Zbog velikih konstantnih faktora u vremenu izvođenja, svi algoritmi s boljim asimptotskim vremenom rada od Strassenova algoritma su nepraktični.<br><br>

Cohn i suradnici stavili su metode kao što su Strassenov i Coppersmith – Winograd algoritam u potpuno drugačiji teoretski kontekst grupe, koristeći trojke podskupova konačnih grupa koje zadovoljavaju svojstvo disjunktnosti zvanu trostruko svojstvo proizvoda (TPP). Oni pokazuju da, ako obitelji proizvoda od vijenaca abelovskih skupina s simetričnim skupinama ostvaruju obitelji trojki podskupova s istovremenom verzijom TPP-a, onda postoje matrični algoritmi množenja s kvadratnom složenošću.Većina istraživača vjeruje da je to doista slučaj. Međutim, Alon, Shpilka i Chris Umans nedavno su pokazali da su neke od tih pretpostavki koje impliciraju brzo matrično množenje nespojive s drugom vjerojatnom pretpostavkom, pretpostavkom suncokreta(suncokret je skup skupova čiji je presjek u parovima konstantan. Ovo konstantno sjecište naziva se jezgra suncokreta).

# 5. Paralelni i distribuirani algoritmi<br>
## Paralelizam dijeljene memorije <br>

Algoritam podijeli pa vladaj, opisan ranije, može se paralelizirati na dva načina za višeprocesore s dijeljenom memorijom. To se temelji na činjenici da se osam rekurzivnih množenja matrica u <br><br>
$$\begin{pmatrix} a_{11}b_{11}+a_{12}b_{21} & a_{11}b_{12}+a_{12}b_{22}\\ a_{21}b_{11}+a_{22}b_{21} & a_{21}b_{12}+a_{22}b_{22} \end{pmatrix}$$ <br>

može izvoditi nezavisno jedan od drugoga, kao i četiri sumiranja. 
Iskorištavajući potpuni paralelizam problema, dobivamo algoritam koji se može izraziti u pseudokodu stila fork-join. 
U paralelnom računanju, fork-join model je način postavljanja i izvršavanja paralelnih programa, tako da se izvršenje paralelno odvaja na određenim točkama u programu, i da se "spoji" (merge) na sljedećoj točki i nastavi sekvencijalno izvršenje.No, algoritam nije praktičan zbog troškova komunikacije koji su svojstveni pomicanju podataka prema i iz privremene matrice T, ali praktičnija varijanta postiže, $Θ(n^2)$ ubrzavanje, bez korištenja privremene matrice<br>

## Izbjegavanje komunikacije i distribuirani algoritmi <br>


Na suvremenim arhitekturama s hijerarhijskom memorijom, trošak učitavanja i pohranjivanja ulaznih elemenata matrice teži da dominira troškovima aritmetike. Na jednom stroju to je količina podataka koja se prenosi između RAM-a i predmemorije, dok je na višestrukom stroju s distribuiranom memorijom to količina prenesena između čvorova; u oba slučaja to se naziva komunikacijska propusnost ( communication bandwidth ). Naivni algoritam koristi tri ugniježđene petlje te mu je komunikacijska propusnost jednaka $\Omega(n^3)$.<br><br>

<b>Cannonov algoritam</b>, također poznat kao 2D algoritam, je algoritam koji izbjegava komunikaciju i koji dijeli svaku ulaznu matricu u blok matricu čiji su elementi podmatrice veličine $\sqrt{\frac{M}{3}} \times \sqrt{\frac{M}{3}} $ , gdje je M veličina brze memorije.<br>
Naivni algoritam tada se koristi preko blok-matrica, izračunavajući proizvode submatrica u cijelosti u brzoj memoriji te na taj način smanjuje communication bandwidth na $O(\frac{n^3}{\sqrt{M}})$<br>

U distribuiranoj postavci s p procesorima raspoređenim u $\sqrt{p} \times \sqrt{p}$ pomoću 2D mreže, jedna podmatrica rezultata može se dodijeliti svakom procesoru, a produkt se može izračunati sa svakim procesorom koji prenosi $O (n^2 / p)$ riječi, što je asimptotski optimalno uz pretpostavku da svaki čvor pohranjuje minimalne $O (n^2 / p)$ elemenata. To se može poboljšati 3D algoritmom, koji raspoređuje procesore u 3D kockastoj mreži, dodjeljujući svaki produkt dvije ulazne podmatrice jednom procesoru. Tada se  rezultantne podmatrice dobiju primjenom redukcije po retcima. Ovaj algoritam prenosi $O (n^2 / p^{2 / 3})$ riječi po procesoru, što je asimptotski optimalno. Međutim, to zahtijeva repliciranje svakog elementa ulazne matrice $p^{1 / 3}$ puta, te stoga zahtijeva faktor od $p{1 / 3}$ više memorije nego što je potrebno za pohranu ulaza. Ovaj algoritam može se kombinirati s Strassenom kako bi se dodatno smanjilo vrijeme izvođenja.<br>
Glavna prednost Cannon algoritma je da zahtjevi za pohranjivanjem ostaju nepromijenjeni i neovisni o broju procesora.
Algoritam skalabilnog univerzalnog matričnog množenja <b>(SUMMA)</b> praktičniji je algoritam koji zahtijeva manje radnog prostora i prevladava potrebu za kvadratnom 2D mrežom.
"2.5D" algoritmi pružaju kontinuirani kompromis između upotrebe memorije i komunikacijske propusnosti. Na suvremenim distribuiranim računalnim okruženjima kao što je <b>MapReduce</b> razvijeni su specijalizirani algoritmi množenja.
## Algoritmi za mreže<br>
Postoji mnoštvo algoritama za množenje na mrežama. Za množenje dva n × n na standardnoj dvodimenzionalnoj mreži pomoću 2D Cannonova algoritma, može se dovršiti množenje u 3n-2 koraka iako je to smanjeno na polovicu tog broja za ponovljene izračune. Standardni niz je neučinkovit jer podaci iz dviju matrica ne stižu istodobno i moraju biti ispunjeni nulama.
<br>
Rezultat je još brži na dvoslojnoj unakrsno ožičenoj mreži, gdje je potrebno samo  2n-1 koraka . Učinkovitost se dodatno poboljšava za ponovljene izračune koji dovode do 100% učinkovitosti. Unakrsno ožičena mreža može se promatrati kao poseban slučaj ne-planarne (tj. višeslojne) procesne strukture.

# ZAKLJUČAK<br>
<img src="slike/pic17.PNG" width="400" height="500" /> <br>
Vidjeli smo da je Strassenov algoritam bio asimptotski brži od naivnog množenja matrica. Od 1969. godine kada je Strassen predstavio svoj rad, dogodile su se brojne i značajne promjene te razvili efikasniji i brži algoritmi za množenje matrica.<br>

Ipak, zbog velikih konstantnih faktora u vremenu izvođenja, svi algoritmi s boljim asimptotskim vremenom rada od Strassenova algoritma su nepraktični.U praksi, Strassenov algoritam nadmašuje jednostavni iterativni algoritam kada dimenzija n postane veća od 100.<br>

Ukratko, čak i ako algoritmi poput Strassenovih imaju niže asimptotičke brzine, oni se možda neće provesti u praksi zbog problema s numeričkom stabilnošću. Međutim, u određenim scenarijima još uvijek mogu biti praktične primjene za njih, kao što je primjena Strassenove metode kada se radi o umnožavanju velikih, gustih matrica određene veličine.
<br><br>
Dakle, naivni algoritam vjerojatno će nam biti najbolji izbor, osim ako su naše matrice vrlo velike!
